In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [2]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [3]:
!ls

dw.ipynb  in.ipynb  pw-Copy1.ipynb  pw-data  pw.ipynb


In [4]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

mkdir: cannot create directory ‘pw-data’: File exists
File ‘./pw-data/201701scripts_sample.json.gz’ already there; not retrieving.

File ‘./pw-data/practices.json.gz’ already there; not retrieving.



In [5]:
!ls

dw.ipynb  in.ipynb  pw-Copy1.ipynb  pw-data  pw.ipynb


## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [6]:
import gzip
import simplejson as json

In [7]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)
    


This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [8]:
print(len(scripts))
scripts[:2]

382726


[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500}]

A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [9]:
print(len(practices))
practices[:2]

12020


[{'code': 'A81001',
  'name': 'THE DENSHAM SURGERY',
  'addr_1': 'THE HEALTH CENTRE',
  'addr_2': 'LAWSON STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 1HU'},
 {'code': 'A81002',
  'name': 'QUEENS PARK MEDICAL CENTRE',
  'addr_1': 'QUEENS PARK MEDICAL CTR',
  'addr_2': 'FARRER STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 2AW'}]

In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`). Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [10]:
import statistics as stats

def quartile(n, pos):
    return pos/4*(n+1)

def describe(key):
    values_arr = sorted([script[key] for script in scripts])
    values_size = len(values_arr)
    total = sum(values_arr)
    avg = total/values_size
    s = stats.stdev(values_arr)
    q25 = values_arr[int(quartile(values_size, 1))]
    med = stats.median(values_arr)
    q75 = values_arr[int(quartile(values_size, 3))]

    return (total, avg, s, q25, med, q75)

In [11]:
# dir(stats)

In [12]:
describe('items')

(4410054, 11.522744731217633, 33.11220959819492, 1, 3.0, 8)

In [13]:
summary = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]

In [14]:
grader.score.pw__summary_statistics(summary)

Your score:  1.0


## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within in each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

In [15]:
bnf_names = list(set([script['bnf_name'] for script in scripts]))
print(len(bnf_names))
print(bnf_names[:10])
assert(len(bnf_names) == 5619)

5619
['LoFric Hydro Nelaton Male NonPVC 8-18(20', 'Psoriderm_Scalp Lot', 'Amoxicillin_Oral Susp Paed 125mg/1.25ml', 'Ebesque XL_Tab 300mg', 'Buscopan Cramps_Tab 10mg', 'Forticreme Complete_Dessert (Banana)', 'Aripiprazole_Tab 10mg', 'Oxycodone HCl_Tab 120mg M/R', 'Pioglitazone/Metformin HCl_Tab15mg/850mg', 'Convatec_Consecura L/Prof Closed Pch Opq']


We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [16]:
groups = {name: [] for name in bnf_names}
# count = 0
# for group in groups:
#     if count == 4:
#         break
#     count += 1
#     print(group, groups[group])

for script in scripts:
    # INSERT ...
    groups[script['bnf_name']].append(script)
#     groups[script['bnf_name']].append(script['bnf_name'])
count = 0
for group in groups:
    if count == 4:
        break
    count += 1
    print(group, groups[group][:4])
# print(groups)

LoFric Hydro Nelaton Male NonPVC 8-18(20 [{'bnf_code': '21020000969', 'items': 3, 'practice': 'L84033', 'bnf_name': 'LoFric Hydro Nelaton Male NonPVC 8-18(20', 'nic': 530.32, 'act_cost': 490.95, 'quantity': 14}, {'bnf_code': '21020000969', 'items': 1, 'practice': 'L84058', 'bnf_name': 'LoFric Hydro Nelaton Male NonPVC 8-18(20', 'nic': 189.4, 'act_cost': 175.34, 'quantity': 5}]
Psoriderm_Scalp Lot [{'bnf_code': '1309000C0BDAAAE', 'items': 1, 'practice': 'P81020', 'bnf_name': 'Psoriderm_Scalp Lot', 'nic': 4.74, 'act_cost': 4.4, 'quantity': 250}, {'bnf_code': '1309000C0BDAAAE', 'items': 1, 'practice': 'Y04972', 'bnf_name': 'Psoriderm_Scalp Lot', 'nic': 4.74, 'act_cost': 4.4, 'quantity': 250}]
Amoxicillin_Oral Susp Paed 125mg/1.25ml [{'bnf_code': '0501013B0AAAIAI', 'items': 1, 'practice': 'L84056', 'bnf_name': 'Amoxicillin_Oral Susp Paed 125mg/1.25ml', 'nic': 3.18, 'act_cost': 2.96, 'quantity': 20}, {'bnf_code': '0501013B0AAAIAI', 'items': 1, 'practice': 'Y05095', 'bnf_name': 'Amoxicillin_

Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

In [17]:
def sum_items(group):
    total = 0
    for item in group:
        total += item["items"]
    return total

items_count = [(group, sum_items(groups[group])) for group in groups]
max_item = [("", 0)]
for item in items_count:
    if item[1] > max_item[0][1]:
        max_item[0] = item
print(max_item)

[('Omeprazole_Cap E/C 20mg', 113826)]


In [25]:
print(len(items_count))
print(items_count[0])

max(sorted(items_count))

5619
('Co-Magaldrox_Susp 195mg/220mg/5ml S/F', 86)


('palmdoc (Reagent)_Strips', 6)

**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [26]:
grader.score.pw__most_common_item(max_item)

Your score:  1.0


**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name')` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [27]:
def group_by_field(data, fields):
    groups = {}
    for item in data:
        group_by = ''
        if len(fields) == 1:
            group_by = item[fields[0]]
        else:
            field_list = []
            for field in fields:
                field_list.append(item[field])
            group_by = tuple(field_list)
            field_list = []
        if group_by in groups:
            groups[group_by].append(item)
        else:
            groups[group_by] = [item]
#     print(groups)
    return groups

In [28]:
groups = group_by_field(scripts, ('bnf_name',))
# test_max_item = ...

def sum_items(group):
    total = 0
    for item in group:
        total += item["items"]
    return total

items_count = [(group, sum_items(groups[group])) for group in groups]
test_max_item = [("", 0)]
for item in items_count:
    if item[1] > test_max_item[0][1]:
        test_max_item[0] = item
print(test_max_item, max_item)
# count = 0
# for group in groups:
#     if count == 4:
#         break
#     print(group, len(groups[group]), '????????????????')
#     count += 1
assert test_max_item == max_item

[('Omeprazole_Cap E/C 20mg', 113826)] [('Omeprazole_Cap E/C 20mg', 113826)]


## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices'`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [29]:
print('SCRIPTS')
print(len(scripts))
scripts[:2]

SCRIPTS
382726


[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500}]

In [30]:
print('PRACTICES')
practices[:2]

PRACTICES


[{'code': 'A81001',
  'name': 'THE DENSHAM SURGERY',
  'addr_1': 'THE HEALTH CENTRE',
  'addr_2': 'LAWSON STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 1HU'},
 {'code': 'A81002',
  'name': 'QUEENS PARK MEDICAL CENTRE',
  'addr_1': 'QUEENS PARK MEDICAL CTR',
  'addr_2': 'FARRER STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 2AW'}]

In [31]:
codes = [item['code'] for item in practices]

print (len(codes))
print(len(set(codes)))

12020
10843


In [32]:
post_codes = [item['post_code'] for item in practices]

print (len(post_codes))
print(len(set(post_codes)))

12020
8306


In [33]:
# scripts['practices'] == practices['code']

practice_postal = {}
for practice in practices:
    if practice['code'] in practice_postal:
        if practice['post_code'] < practice_postal[practice['code']]:
            practice_postal[practice['code']] = practice['post_code']
    else:
        practice_postal[practice['code']] = practice['post_code']

count = 0
for i in practice_postal:
    print(i, practice_postal[i])
    if count == 5:
        break
    count += 1

A81001 TS18 1HU
A81002 TS18 2AW
A81003 TS25 1QU
A81004 TS1 3BE
A81005 TS14 7DJ
A81006 TS18 2AT


**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [105]:
practice_postal['K82019']

'HP21 8TR'

In [106]:
assert practice_postal['K82019'] == 'HP21 8TR'

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [46]:
practice_by_code = group_by_field(practices, ('code',))

# print(len(practices_by_postal))
practice_postal = {}
for practice_code in practice_by_code:
    practice_postal[practice_code] = practice_by_code[practice_code][0]['post_code']
    for practice in practice_by_code[practice_code]:
#         practice_postal[practice_code] = max(practice['post_code'], practice_postal[practice_code])
        if practice['post_code'] < practice_postal[practice_code]:
            practice_postal[practice_code] = practice['post_code']
            
count = 0
for i in practice_postal:
    print(i, practice_postal[i])
    if count == 5:
        break
    count += 1

practice_postal['K82019']

A81001 TS18 1HU
A81002 TS18 2AW
A81003 TS25 1QU
A81004 TS1 3BE
A81005 TS14 7DJ
A81006 TS18 2AT


'HP21 8TR'

In [47]:
assert practice_postal['K82019'] == 'HP21 8TR'

Now we can join `practice_postal` to `scripts`.

In [48]:
joined = scripts[:]
for script in joined:
    script['post_code'] = practice_postal[script['practice']]

In [49]:
joined[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000,
  'post_code': 'SK11 6JL'},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500,
  'post_code': 'SK11 6JL'}]

Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [50]:
group_by_postal = group_by_field(joined, ('post_code',))

In [51]:
items_by_post = {}

for postal_code in group_by_postal:
    items_by_post[postal_code] = 0
    for script in group_by_postal[postal_code]:
        items_by_post[postal_code] += script['items']
        
# for script in joined:
#     if script['post_code'] in items_by_post:
#         items_by_post[script['post_code']] += script['items']
#     else:
#         items_by_post[script['post_code']] = script['items']

In [52]:
count = 0
for i in items_by_post:
    print(i, items_by_post[i])
    if count == 5:
        break
    count += 1

SK11 6JL 110071
CW5 5NX 38797
CW1 3AW 64104
CW7 1AT 43164
CH65 6TG 25090
CH1 4DS 34915


In [114]:
print(len(postal_totals))

100


In [53]:
postal_totals = sorted([(item, items_by_post[item]) for item in items_by_post])
print(len(postal_totals))
postal_totals = sorted([(item, items_by_post[item]) for item in items_by_post])[:100]
print(len(postal_totals))

118
100


In [54]:
grader.score.pw__postal_totals(postal_totals)

Your score:  1.0


## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

Now we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. Use the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

In [55]:
total_items_by_bnf_post = {}
for script in joined:
    if (script['post_code'], script['bnf_name']) in total_items_by_bnf_post:
        total_items_by_bnf_post[(script['post_code'], script['bnf_name'])] += script['items']
    else:
        total_items_by_bnf_post[(script['post_code'], script['bnf_name'])] = script['items']

# total_items_by_bnf_post = ...
assert len(total_items_by_bnf_post) == 141196

In [56]:
count = 0
for i in total_items_by_bnf_post:
    print(i, total_items_by_bnf_post[i])
    if count == 5:
        break
    count += 1

('SK11 6JL', 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F') 5
('SK11 6JL', 'Alginate_Raft-Forming Oral Susp S/F') 3
('SK11 6JL', 'Sod Algin/Pot Bicarb_Susp S/F') 94
('SK11 6JL', 'Sod Alginate/Pot Bicarb_Tab Chble 500mg') 9
('SK11 6JL', 'Gaviscon Infant_Sach 2g (Dual Pack) S/F') 41
('SK11 6JL', 'Gaviscon Advance_Liq (Aniseed) (Reckitt)') 98


In [57]:
# total_items_by_bnf_post = group_by_field(joined, ('post_code', 'bnf_name'))
# len(total_items_by_bnf_post)
total_items_by_bnf_post[('NE24 1DX', 'Adalat Ret_Tab 20mg')]

18

In [72]:
# count = 0
# for i in total_items_by_bnf_post:
#     print(i, total_items_by_bnf_post[i])
#     if count == 5:
#         break
#     count += 1
# total_items_by_bnf_post.items()
my_dict = {'a': 1, 'b': 2, 'c': 3 }
print(my_dict.values())
print(my_dict.keys())
print(my_dict.items())
print(list(my_dict.items()))

dict_values([1, 2, 3])
dict_keys(['a', 'b', 'c'])
dict_items([('a', 1), ('b', 2), ('c', 3)])
[('a', 1), ('b', 2), ('c', 3)]


Let's use `total_by_item_post` to find the maximum item total for each postal code. To do this, we will want to regroup `total_by_item_post` by `'post_code'` only, not by `('post_code', 'bnf_name')`. First let's turn `total_by_item_post` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in `total_by_item_post` after grouping it by `'post_code'`.

In [62]:
total_items = []
for (post_code, bnf_name), total in list(total_items_by_bnf_post.items()): 
    total_items.append({
        'post_code': post_code,
        'bnf_name': bnf_name,
        'total': total
    })

len(total_items)

141196

In [63]:
# total_items_by_post = joined[:]
# for script in joined:
#         total_items_by_post.append(script)

total_items_by_post = group_by_field(total_items, ('post_code',))

len(total_items_by_post)

118

In [64]:
# total_items_by_post = {}
# for script in joined:
#     if script['post_code'] in total_items_by_post:
#         total_items_by_post[script['post_code']] += script['items']
#     else:
#         total_items_by_post[script['post_code']] = script['items']

# total_items = ...
assert len(total_items_by_post) == 118

In [65]:
count = 0
for i in total_items_by_post:
    print(i, total_items_by_post[i][:3])
    if count == 4:
        break
    count += 1

SK11 6JL [{'post_code': 'SK11 6JL', 'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F', 'total': 5}, {'post_code': 'SK11 6JL', 'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F', 'total': 3}, {'post_code': 'SK11 6JL', 'bnf_name': 'Sod Algin/Pot Bicarb_Susp S/F', 'total': 94}]
CW5 5NX [{'post_code': 'CW5 5NX', 'bnf_name': 'Sod Algin/Pot Bicarb_Susp S/F', 'total': 21}, {'post_code': 'CW5 5NX', 'bnf_name': 'Sod Alginate/Pot Bicarb_Tab Chble 500mg', 'total': 4}, {'post_code': 'CW5 5NX', 'bnf_name': 'Gaviscon_Liq Orig Aniseed Relief', 'total': 3}]
CW1 3AW [{'post_code': 'CW1 3AW', 'bnf_name': 'Maalox_Susp 195mg/220mg/5ml S/F', 'total': 2}, {'post_code': 'CW1 3AW', 'bnf_name': 'Sod Alginate/Pot Bicarb_Tab Chble 500mg', 'total': 2}, {'post_code': 'CW1 3AW', 'bnf_name': 'Gaviscon_Liq Orig Aniseed Relief', 'total': 2}]
CW7 1AT [{'post_code': 'CW7 1AT', 'bnf_name': 'Sod Algin/Pot Bicarb_Susp S/F', 'total': 53}, {'post_code': 'CW7 1AT', 'bnf_name': 'Gaviscon Infant_Sach 2g (Dual Pack) S/F', 'tota

In [66]:
from operator import itemgetter

In [67]:
get_total = itemgetter('total')
get_total(total_items_by_post['SK11 6JL'][2])

94

Now we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the item total and `'bnf_name'`.

In [68]:
test_list = total_items_by_post['SK11 6JL']
sorted(test_list, key=itemgetter('total'), reverse=True)

[{'post_code': 'SK11 6JL',
  'bnf_name': 'Omeprazole_Cap E/C 20mg',
  'total': 3219},
 {'post_code': 'SK11 6JL',
  'bnf_name': 'Aspirin Disper_Tab 75mg',
  'total': 1955},
 {'post_code': 'SK11 6JL',
  'bnf_name': 'Lansoprazole_Cap 30mg (E/C Gran)',
  'total': 1952},
 {'post_code': 'SK11 6JL', 'bnf_name': 'Simvastatin_Tab 40mg', 'total': 1908},
 {'post_code': 'SK11 6JL', 'bnf_name': 'Paracet_Tab 500mg', 'total': 1564},
 {'post_code': 'SK11 6JL', 'bnf_name': 'Ramipril_Cap 10mg', 'total': 1449},
 {'post_code': 'SK11 6JL', 'bnf_name': 'Atorvastatin_Tab 20mg', 'total': 1349},
 {'post_code': 'SK11 6JL', 'bnf_name': 'Amlodipine_Tab 5mg', 'total': 1301},
 {'post_code': 'SK11 6JL',
  'bnf_name': 'Metformin HCl_Tab 500mg',
  'total': 1186},
 {'post_code': 'SK11 6JL',
  'bnf_name': 'Ventolin_Evohaler 100mcg (200 D)',
  'total': 1117},
 {'post_code': 'SK11 6JL', 'bnf_name': 'Amlodipine_Tab 10mg', 'total': 1116},
 {'post_code': 'SK11 6JL',
  'bnf_name': 'Levothyrox Sod_Tab 25mcg',
  'total': 1071},

In [74]:
max_item_by_post = []
groups = list(total_items_by_post.values())

for group in groups:
    max_total = sorted(group, key=itemgetter('total'), reverse=True)[0]
    max_item_by_post.append(max_total)

In [75]:
max_item_by_post[:3]

[{'post_code': 'SK11 6JL',
  'bnf_name': 'Omeprazole_Cap E/C 20mg',
  'total': 3219},
 {'post_code': 'CW5 5NX',
  'bnf_name': 'Omeprazole_Cap E/C 20mg',
  'total': 1419},
 {'post_code': 'CW1 3AW',
  'bnf_name': 'Omeprazole_Cap E/C 20mg',
  'total': 2364}]

In [76]:
max_item_by_post = [sorted(group, key=itemgetter('total'), reverse=True)[0] for group in groups]

In [77]:
max_item_by_post[:3]

[{'post_code': 'SK11 6JL',
  'bnf_name': 'Omeprazole_Cap E/C 20mg',
  'total': 3219},
 {'post_code': 'CW5 5NX',
  'bnf_name': 'Omeprazole_Cap E/C 20mg',
  'total': 1419},
 {'post_code': 'CW1 3AW',
  'bnf_name': 'Omeprazole_Cap E/C 20mg',
  'total': 2364}]

In [78]:
len(max_item_by_post)

118

In [79]:
# max_item_by_post = {}

# for postal_code in total_items_by_post:
#     max_item = 0
#     last_bnf_name = ''
# #     print(total_items_by_post[postal_code])
#     for script in total_items_by_post[postal_code]:
#         last_bnf_name = script['bnf_name']
# #         print(script)
#         if script['items'] > max_item:
#             max_item = script['items']
#         elif max_item == script['items']:
#             if last_bnf_name < script['bnf_name']:
#                 last_bnf_name = script['bnf_name']
#                 max_item = script['items']
#     max_item_by_post[postal_code] = max_item
            
        

In [80]:
# count = 0
# for i in max_item_by_post:
#     print(i)
#     if count == 4:
#         break
#     count += 1

In order to express the item totals as a proportion of the total amount of items prescribed across all `'bnf_name'` in a postal code, we'll need to use the total items prescribed that previously calculated as `items_by_post`. Calculate the proportions for the most common `'bnf_names'` for each postal code. Format your answer as a list of tuples: `[(post_code, bnf_name, total)]`

In [81]:
print(len(items_by_post))
print(len(max_item_by_post))

118
118


In [82]:
items_by_region = max_item_by_post[:100]
# for postal_code in max_item_by_post:
#     for script in total_items_by_post[postal_code]:
#         max_item = 0
#         bnf_name = 
#     items_by_region.append((postal_code, 'Salbutamol_Inha 100mcg (200 D) CFF', max_item_by_post[postal_code]/items_by_post[postal_code]))
#     items_by_region.append((postal_code, '', items_by_post[postal_code]/max_item_by_post[postal_code]))
# items_by_region = [('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.0341508247)] * 100

In [83]:

items_by_region = []
for item in max_item_by_post:
#     print(item)
    post_code = item['post_code']
    bnf_name = item['bnf_name']
    total = item['total']
    proportion = total/items_by_post[post_code]
    items_by_region.append((post_code, bnf_name, proportion))
    

In [84]:
items_by_region = sorted(items_by_region)[:100]
len(items_by_region)

100

In [85]:
grader.score.pw__items_by_region(items_by_region)

Your score:  1.0


*Copyright &copy; 2017 The Data Incubator.  All rights reserved.*